$$\Huge\textbf{Decision Tree Models}$$

# Imports

In [ ]:
# Standard library imports
import os  # directory and file operations
import shutil  # copying files
import time  # adding delays

# Installed library imports
import cv2  #  OpenCV for computer vision tasks
import numpy as np  # numerical operations
import matplotlib.pyplot as plt  # plotting and visualization
import pandas as pd
from PIL import Image  # image processing
import seaborn as sns  # confusion matrix visualization
from skimage import exposure  #  scikit-image for image processing tasks
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report  # evaluation metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split  # splitting datasets and cross-validation
from sklearn.tree import DecisionTreeClassifier  # decision tree classifier
import torchvision.transforms as transforms  # data augmentation
from sklearn.preprocessing import StandardScaler #
from sklearn.pipeline import Pipeline
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm

# Global Constansts

In [ ]:
# constants
NUM_IMAGES_PER_CLASS = 1000
TEST_SPLIT_PERCENTAGE = 0.15
TRAIN_VAL_SPLIT_PERCENTAGE = 1 - TEST_SPLIT_PERCENTAGE  # the rest is for training and validation
BATCH_SIZE = 100  # number of images to copy at once
RANDOM_SEED = 42  # seed for reproducibility

IMAGE_SIZE = (256, 256)
VALIDATION_SPLIT = 15 / 85  # train_val is 85% of the total dataset, we want the validation set to be 15% of the total dataset
CLASSES = ['airplane_cabin', 'hockey_arena', 'movie_theater', 'staircase', 'supermarket']

In [ ]:
# data directories
PROJECT_DATASET_DIR = '/content/drive/MyDrive/project_dataset'
PROJECT_OUTPUT_DIR = '/content/drive/MyDrive/project_output'
CLASSES_ORIGINAL = ['airplane_cabin', 'hockey_arena', 'movie_theater', 'staircase', 'supermarket']
CLASSES_NEW = ['airplane_cabin', 'hockey_arena', 'movie_theater', 'staircase', 'supermarket']

TRAIN_VAL_DIR = os.path.join(PROJECT_DATASET_DIR, 'train_val')
TEST_DIR = os.path.join(PROJECT_DATASET_DIR, 'test')

TRAIN_DIR = os.path.join(PROJECT_DATASET_DIR, 'train')  # new directory
VALIDATION_DIR = os.path.join(PROJECT_DATASET_DIR, 'validation')  # new directory
CROSS_VALI_DIR = os.path.join(PROJECT_DATASET_DIR, 'cross_validation') # new directory for cross_validation

In [ ]:
# please do not remove it, it's for test purpose!!

# processed data directories: case without cross validation
PROCESSED_DATA_DIR = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_dataset\processed_data'
TEST_DIR = os.path.join(PROCESSED_DATA_DIR, 'without_cross_validation', 'test')
TRAIN_DIR = os.path.join(PROCESSED_DATA_DIR, 'without_cross_validation','train') 
VALIDATION_DIR = os.path.join(PROCESSED_DATA_DIR, 'without_cross_validation', 'validation')  # new directory

# project output directory
PROJECT_OUTPUT_DIR = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_output'
output_path = r"C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_output\eval_results_grid_search.txt"
output_path_filetest = r"C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_output\eval_results_filetest_pca.txt"



# 1. Help Functions

## 1.1. Load image data and labels for each dataset.


*   Number of training data: 14000
*   Number of validation/test data: 750





In [ ]:
def load_images_and_labels(base_dir):
    """
    Load image data and labels from a specified directory. This function traverses each class directory, loads each image, and assigns labels based on the directory name.

    Inputs:
    - base_dir: A string, the path to the directory containing subdirectories for each class.

    Outputs:
    - A tuple containing two numpy arrays:
      1. `data`: an array of flattened image data.
      2. `labels`: an array of labels corresponding to the images.

    Each class is expected to have its own subdirectory within `base_dir` with the name of the class.
    """
    # Initialize lists to store image data and labels
    data = []
    labels = []

    # Define the classes and create a mapping from class names to a unique integer
    classes = ['airplane_cabin', 'hockey_arena', 'movie_theater', 'staircase', 'supermarket']
    class_labels = {cls_name: idx for idx, cls_name in enumerate(classes)}

    # Iterate over each class directory in the base directory
    for cls_name in classes:
        class_dir = os.path.join(base_dir, cls_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = Image.open(img_path)  # Open the image directly
            img_data = np.array(img).flatten()  # Flatten the image data into a one-dimensional vector
            data.append(img_data)
            labels.append(class_labels[cls_name])

    # Convert the lists of data and labels to numpy arrays and return
    return np.array(data), np.array(labels)

## 1.2. Generate CSV file

In [ ]:
def generate_csv_filename(max_depth, min_samples_split, min_samples_leaf, criterion):
    """
    Generate a filename for storing CSV data based on the parameters of a decision tree model.

    Parameters:
    - max_depth (int or None): The maximum depth of the tree.
    - min_samples_split (int): The minimum number of samples required to split an internal node.
    - min_samples_leaf (int): The minimum number of samples required to be at a leaf node.
    - criterion (str): The function to measure the quality of a split.

    Returns:
    - str: A string representing the filename.
    """
    # Replace None with 'None' for filename compatibility
    max_depth_str = 'None' if max_depth is None else max_depth

    # Construct the filename
    filename = f"d{max_depth_str}_s{min_samples_split}_l{min_samples_leaf}_{criterion}.csv"

    return filename

# 2. Evaluation Metrics

In [ ]:
def evaluate_model(predictions, true_labels):
    """
    Evaluate the predictions of a model using the true labels.
    Calculate the accuracy, precision, recall, and confusion matrix of the predictions.

    Inputs:
    - predictions (np.ndarray): The predicted labels from the model.
    - true_labels (np.ndarray): The true labels for the validation data.

    Outputs:
    - accuracy (float): The accuracy of the model on the validation data.
    - precision (float): The precision of the model on the validation data.
    - recall (float): The recall of the model on the validation data.
    - f1 (float): The f1-score of the model on the validation data.
    - conf_matrix (np.ndarray): The confusion matrix of the model's predictions.
    - report (str): The classification report of the model's predictions.
    """
    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    
    # Calculate precision (macro-average for multi-class classification)
    precision = precision_score(true_labels, predictions, average='macro')
    
    # Calculate recall (macro-average for multi-class classification)
    recall = recall_score(true_labels, predictions, average='macro')
    
    # Calculate f1-score (macro-average for multi-class classification)
    f1 = f1_score(true_labels, predictions, average='macro')

    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predictions)

    # Generate classification report
    report = classification_report(true_labels, predictions)

    return accuracy, precision, recall, f1, conf_matrix, report


In [ ]:
def print_evaluation_results(results):
    """
    Print and plot the evaluation results.

    Inputs:
    - results (tuple): A tuple containing the following evaluation results:
        - accuracy (float): The accuracy of the model.
        - precision (float): The precision of the model.
        - recall (float): The recall of the model.
        - f1 (float): The f1-score of the model.
        - conf_matrix (np.ndarray): The confusion matrix of the model's predictions.
        - report (str): The classification report of the model's predictions.

    No outputs. The function prints the evaluation metrics and plots the confusion matrix.
    """
    # Unpack the results tuple
    accuracy, precision, recall, f1, conf_matrix, report = results

    # Print the evaluation metrics
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-score: {f1}')

    print("Classification Report:")
    print(report)

    # Print the confusion matrix
    print("Confusion Matrix:")
    print(conf_matrix)
    
    # Plot the confusion matrix (commented out for now)
    #plt.figure(figsize=(10, 7))
    #sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
    #            xticklabels=CLASSES, yticklabels=class_labels)
    #plt.xlabel('Predicted')
    #plt.ylabel('Actual')
    #plt.title('Confusion Matrix')
    #plt.show()


In [ ]:
def save_evaluation_results_to_txt(results, filename, hyperparameters, dataset):
    """
    Save evaluation results to a text file.

    Parameters:
    - results (tuple): A tuple containing accuracy, precision, recall, f1, confusion matrix, and classification report.
    - filename (str): Full path of the text file to save the results.
    - hyperparameters (dict): The hyperparameters used for the model.
    - dataset (str): The dataset on which the model was evaluated ('Training' or 'Validation').
    """
    accuracy, precision, recall, f1, conf_matrix, report = results

    with open(filename, 'a') as file:
        file.write(f"Evaluation Results on {dataset} Data:\n")
        for key, value in hyperparameters.items():
            file.write(f"{key}: {value}\n")
        file.write(f"Accuracy: {accuracy}\n")
        file.write(f"Precision: {precision}\n")
        file.write(f"Recall: {recall}\n")
        file.write(f"F1 Score: {f1}\n")
        file.write(f"Confusion Matrix:\n{conf_matrix}\n")
        file.write(f"Classification Report:\n{report}\n")
        file.write("\n")

# 3. Additional Data Preprocessing on Training Set

## 3.1. Feature Reduction: PCA

In [ ]:
def determine_pca_components(features, variance_threshold=0.95):
    """
    Determine the number of PCA components needed to explain a given amount of variance.

    Inputs:
    - features (np.ndarray): Concatenated feature vectors.
    - variance_threshold (float): The amount of variance to be explained by the selected components.

    Outputs:
    - n_components (int): The number of components to keep.
    """
    pca = PCA()
    pca.fit(features)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.searchsorted(cumulative_variance, variance_threshold) + 1
    return n_components

In [ ]:
def apply_pca_to_features(features, pca_model):
    """
    Apply PCA to the concatenated feature vectors to reduce dimensionality.

    Inputs:
    - features (np.ndarray): Concatenated feature vectors.
    - pca_model (PCA): Fitted PCA model.

    Outputs:
    - pca_features (np.ndarray): Feature vectors after PCA transformation.
    """
    pca_features = pca_model.transform(features)
    return pca_features


In [ ]:
# save pca-transformed data
def save_transformed_data(file_path, data):
    """
    Save the transformed data to a file.

    Inputs:
    - file_path (str): Path to the file where the data will be saved.
    - data (np.ndarray): The data to be saved.
    """
    np.save(file_path, data)

In [ ]:
# load pca transformed data
def load_transformed_data(file_path):
    """
    Load the transformed data from a file.

    Inputs:
    - file_path (str): Path to the file from where the data will be loaded.

    Outputs:
    - data (np.ndarray): The loaded data.
    """
    return np.load(file_path)

## 3.2. Transformation Application

Edge detection, corner detection, contour detection, histogram equalization

In [ ]:
def apply_transformations(image):
    """
    Apply all transformations (edge detection, corner detection, contour detection, histogram equalization, and PCA) to the image and concatenate the features.

    Inputs:
    - image (PIL Image object): Input image.

    Outputs:
    - features (np.ndarray): Concatenated feature vector from all transformations.
    """
    image_np = np.array(image.convert('L'))
    
    # Edge detection
    edges = cv2.Canny(image_np, 100, 200)
    edges_flat = edges.flatten()
    #print(f'Edge detection adds {len(edges_flat)} features.')
    
    # Corner detection
    dst = cv2.cornerHarris(image_np, 2, 3, 0.04)
    dst = cv2.dilate(dst, None)
    corners = (dst > 0.01 * dst.max()).astype(np.uint8) * 255
    corners_flat = corners.flatten()
    #print(f'Corner detection adds {len(corners_flat)} features.')
    
    # Contour detection
    _, threshold = cv2.threshold(image_np, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_image_np = cv2.drawContours(np.zeros_like(image_np), contours, -1, (255), 1)
    contours_flat = contour_image_np.flatten()
    #print(f'Contour detection adds {len(contours_flat)} features.')
    
    # Histogram equalization
    equalized_image_np = exposure.equalize_hist(image_np)
    equalized_flat = equalized_image_np.flatten()
    #print(f'Histogram equalization adds {len(equalized_flat)} features.')
    
    # Flatten all features and concatenate
    features = np.hstack([
        edges_flat,
        corners_flat,
        contours_flat,
        equalized_flat
    ])
    
    #print(f'Total features: {len(features)}')
    return features


In [ ]:
#img_path = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_dataset\processed_data\without_cross_validation\train\airplane_cabin\00000004.jpg'
#img = Image.open(img_path)
#apply_transformations(img)

## 3.3. Data Loading and Transformed Feature Vector Creation

In [ ]:
def load_and_process_images(base_dir):
    """
    Load image data, apply transformations, and concatenate features.

    Inputs:
    - base_dir: A string, the path to the directory containing subdirectories for each class.

    Outputs:
    - data: An array of concatenated features.
    - labels: An array of labels corresponding to the images.
    """
    data = []
    labels = []

    class_labels = {cls_name: idx for idx, cls_name in enumerate(CLASSES)}

    for cls_name in CLASSES:
        class_dir = os.path.join(base_dir, cls_name)
        for img_name in tqdm(os.listdir(class_dir), desc=f'Processing {cls_name}'):
            img_path = os.path.join(class_dir, img_name)
            img = Image.open(img_path)
            features = apply_transformations(img)
            data.append(features)
            labels.append(class_labels[cls_name])

    data = np.array(data)
    labels = np.array(labels)

    return data, labels


# 4. Decision Tree Models

In [ ]:
# Load training, validation, and test data
train_data, train_labels = load_images_and_labels(TRAIN_DIR)
validation_data, validation_labels = load_images_and_labels(VALIDATION_DIR)
test_data, test_labels = load_images_and_labels(TEST_DIR)

## 4.1. Supervised Decision Tree

In [ ]:
# Create a decision tree model
model_dt = DecisionTreeClassifier(max_depth=10, 
                                  min_samples_split=20, 
                                  min_samples_leaf=5, 
                                  criterion='entropy', 
                                  random_state=RANDOM_SEED)

model_dt.fit(train_data, train_labels)

### Test Zone Lu

In [ ]:
# Evaluate the model using training data
train_predictions_dt = model_dt.predict(train_data)
train_accuracy_dt = accuracy_score(train_labels, train_predictions_dt)
print(f'Training Accuracy for model_dt2: {train_accuracy_dt}')

# Evaluate the model using validation data
validation_predictions_dt = model_dt.predict(validation_data)
vali_accuracy_dt = accuracy_score(validation_labels, validation_predictions_dt)
print(f'Validation Accuracy for model_dt2: {vali_accuracy_dt}')

### Test zone Hélène

In [ ]:
## load and process training data
train_data, train_labels = load_and_process_images(TRAIN_DIR)

In [ ]:
## feature reduction with PCA on the training data

# determine the optimal number of PCA components for the training data
#optimal_components = determine_pca_components(train_data)
#print(f'Optimal number of PCA components: {optimal_components}')


In [ ]:
TRAIN_DATA_PCA = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\saved_models\train_data_pca.npy'
VAL_DATA_PCA = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\saved_models\validation_data_pca.npy'
TEST_DATA_PCA = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\saved_models\test_data_pca.npy'

In [ ]:
# apply PCA to the training data
#pca = PCA(n_components=optimal_components)
#train_data_pca = pca.fit_transform(train_data)
#save_transformed_data(TRAIN_DATA_PCA, train_data_pca)

In [ ]:
# apply the same PCA model to validation data
validation_data, validation_labels = load_and_process_images(VALIDATION_DIR)
#validation_data_pca = apply_pca_to_features(validation_data, pca)
#save_transformed_data(VAL_DATA_PCA, validation_data_pca)

In [ ]:
# apply the same PCA model to test data
test_data, test_labels = load_and_process_images(TEST_DIR)
#test_data_pca = apply_pca_to_features(test_data, pca)
#save_transformed_data(TEST_DATA_PCA, test_data_pca)

#### i. Basic DT

In [ ]:
########## test purpose, please do not delete!


##### basic DT using one set of hyperparameters
# create a decision tree model
model_dt = DecisionTreeClassifier(max_depth=8, 
                                  min_samples_split=30, 
                                  min_samples_leaf=10, 
                                  criterion='entropy', 
                                  random_state=RANDOM_SEED)

# train model
model_dt.fit(train_data, train_labels)

# evaluate the model on the validation set
val_predictions = model_dt.predict(validation_data)
val_results = evaluate_model(val_predictions, validation_labels)

# evaluate the model on the training set
train_predictions = model_dt.predict(train_data)
train_results = evaluate_model(train_predictions, train_labels)

# print and plot the evaluation results
print_evaluation_results(train_results)
print_evaluation_results(val_results)

hyperparameters = {
                    'max_depth': 10,
                    'min_samples_split': 20,
                    'min_samples_leaf': 2
                }

# save results in .txt file
save_evaluation_results_to_txt(train_results, output_path_filetest, hyperparameters, 'Training')
save_evaluation_results_to_txt(val_results, output_path_filetest, hyperparameters, 'Validation')


#### ii. Grid Search to Find the Set of the Best Hyperparamers

In [ ]:
# define the parameter grid
param_grid = {
    'max_depth': [8, 10],
    'min_samples_split': [20, 30],
    'min_samples_leaf': [5, 10, 15]
}

##### Method 1: GridSearchCV (too slow)

In [ ]:
## method 1: using GridSearchCV from scikitlearn

# create a general DT classifier
gridcv_model_dt = DecisionTreeClassifier(criterion='entropy', random_state=RANDOM_SEED)

# perform grid search with cross-validation
gridcv_search = GridSearchCV(estimator=gridcv_model_dt, param_grid=param_grid, cv=5, scoring='accuracy')
gridcv_search.fit(train_data, train_labels)

# get the best parameters
best_params = gridcv_search.best_params_

print("Best hyperparameters:", best_params)

# train the model with the best parameters
model_dt = DecisionTreeClassifier(**best_params, criterion='entropy', random_state=RANDOM_SEED)
model_dt.fit(train_data, train_labels)

# use the trained model to predict 
train_predictions_dt = model_dt.predict(train_data)
validation_predictions_dt = model_dt.predict(validation_data)

# evaluate the model using the evaluate_model(predictions, true_labels) function
train_acc_dt, train_prec_dt, train_rec_dt, train_f1_dt, train_conf_matrix_dt, train_report_dt = evaluate_model(train_predictions_dt, train_labels)

val_acc_dt, val_prec_dt, val_rec_dt, val_f1_dt, val_conf_matrix_dt, val_report_dt = evaluate_model(validation_predictions_dt, validation_labels)

# print and plot the evaluation results
print_evaluation_results(train_acc_dt, 
                         train_prec_dt, 
                         train_rec_dt, 
                         train_f1_dt,
                         train_conf_matrix_dt,
                         train_report_dt)

print_evaluation_results(val_acc_dt, 
                         val_prec_dt, 
                         val_rec_dt, 
                         val_f1_dt,
                         val_conf_matrix_dt,
                         val_report_dt)


##### Method 2: Grid Search Without Cross Validation

In [ ]:
def perform_grid_search_without_cv(train_data, train_labels, val_data, val_labels, param_grid, output_path, random_state=RANDOM_SEED):
    """
    Perform grid search without cross-validation to find the best hyperparameters for a Decision Tree classifier.

    Parameters:
    - train_data (array-like): Training data.
    - train_labels (array-like): Labels corresponding to the training data.
    - val_data (array-like): Validation data.
    - val_labels (array-like): Labels corresponding to the validation data.
    - param_grid (dict): Dictionary with parameters names (str) as keys and lists of parameter settings to try as values.
    - output_path (str): Path to save the evaluation results.
    - random_state (int): Random seed for reproducibility.

    Outputs:
    - dict: Best hyperparameters found.
    - float: Best accuracy score achieved.
    """

    best_params = None
    best_score = 0

    # Perform grid search without cross-validation
    for max_depth in param_grid['max_depth']:
        for min_samples_split in param_grid['min_samples_split']:
            for min_samples_leaf in param_grid['min_samples_leaf']:
                # Create a Decision Tree classifier with the current combination of hyperparameters
                model_dt = DecisionTreeClassifier(
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf,
                    criterion='entropy',
                    random_state=random_state
                )
                
                # Train the model
                model_dt.fit(train_data, train_labels)
                
                # Evaluate the model on the validation set
                val_predictions = model_dt.predict(val_data)
                val_results = evaluate_model(val_predictions, val_labels)
                
                # Evaluate the model on the training set
                train_predictions = model_dt.predict(train_data)
                train_results = evaluate_model(train_predictions, train_labels)

                # Save results to text file
                hyperparameters = {
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf
                }

                save_evaluation_results_to_txt(train_results, output_path, hyperparameters, 'Training')
                save_evaluation_results_to_txt(val_results, output_path, hyperparameters, 'Validation')
                
                # Update best parameters if the current score is better
                if val_results[0] > best_score:  # val_results[0] is accuracy
                    best_score = val_results[0]
                    best_params = hyperparameters

    return best_params, best_score



In [ ]:
# perform grid search without cross-validation
best_params, best_score = perform_grid_search_without_cv(train_data, 
                                                         train_labels, 
                                                         validation_data, 
                                                         validation_labels, 
                                                         param_grid, 
                                                         output_path)

print("Best hyperparameters without cross-validation:", best_params)
print("Best accuracy score without cross-validation:", best_score)

### Save Predictions in a CSV

In [ ]:
# Save the labels and predictions to a CSV file
results_df = pd.DataFrame({
    'Actual Labels': validation_labels,
    'Predicted Labels': validation_predictions_dt
})

# File path for the CSV file
filename = generate_csv_filename(10, 20, 2, 'entropy')
csv_file_path = os.path.join(PROJECT_OUTPUT_DIR, 'csvs/'+filename)

# Check if the file exists, if not create one, if yes append without writing headers
if not os.path.isfile(csv_file_path):
    results_df.to_csv(csv_file_path, index=False)
else:
    results_df.to_csv(csv_file_path, mode='a', header=False, index=False)

## 4.2. Supervised Decision Tree with 5-fold validation

There is a directory for cross validation.

In [ ]:
def prepare_cross_validation_dirs(base_dir, output_dir, n_splits=5):
    """
    Prepares cross-validation directories for image data.
    
    Parameters:
    - base_dir: Path to the directory containing the original unnormalized, unaugmented data.
    - output_dir: Path to the directory where the cross-validation splits will be stored.
    - n_splits: Number of cross-validation splits.
    """
    # Create the cross_validation directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Read all images and their labels
    images, labels = [], []
    class_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
    for folder in class_folders:
        folder_path = os.path.join(base_dir, folder)
        for img in os.listdir(folder_path):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                images.append(os.path.join(folder, img))
                labels.append(folder)

    # Prepare stratified splits
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    images = np.array(images)
    labels = np.array(labels)

    # Generate splits and directories
    for fold, (train_idx, val_idx) in enumerate(skf.split(images, labels), 1):
        fold_dir = os.path.join(output_dir, f'fold_{fold}')
        os.makedirs(fold_dir, exist_ok=True)

        # Directories for training and validation inside each fold
        train_dir = os.path.join(fold_dir, 'train')
        val_dir = os.path.join(fold_dir, 'val')
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)

        # Copy training images
        for img in images[train_idx]:
            src = os.path.join(base_dir, img)
            dst = os.path.join(train_dir, img)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy(src, dst)

        # Copy validation images
        for img in images[val_idx]:
            src = os.path.join(base_dir, img)
            dst = os.path.join(val_dir, img)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy(src, dst)

In [ ]:
base_dir = os.path.join(PROJECT_DATASET_DIR, 'train_val') 
output_dir =os.path.join(PROJECT_DATASET_DIR, 'cross_validation') 
prepare_cross_validation_dirs(base_dir, output_dir)

In [ ]:
# here we just temporary use the raw pictures to test †he training of 5-fold
# will modify these when have the processed cross_vali datasets
def load_images_from_folder(folder):
    images = []
    labels = []
    for class_folder in os.listdir(folder):
        class_folder_path = os.path.join(folder, class_folder)
        for filename in os.listdir(class_folder_path):
            img_path = os.path.join(class_folder_path, filename)
            img = imread(img_path)
            images.append(img)
            labels.append(class_folder)
    return np.array(images), np.array(labels)

def train_decision_tree(train_dir, val_dir):
    # Load images and labels
    X_train, y_train = load_images_from_folder(train_dir)
    X_val, y_val = load_images_from_folder(val_dir)

    # Define the decision tree model
    model = Pipeline([
        ('scaler', StandardScaler()),  # Scale images
        ('classifier', DecisionTreeClassifier(max_depth=10, min_samples_split=20, min_samples_leaf=2, criterion='entropy'))
    ])
    # Train the model
    model.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Flatten images for training

    # Calculate training accuracy
    y_train_pred = model.predict(X_train.reshape(X_train.shape[0], -1))
    train_accuracy = accuracy_score(y_train, y_train_pred)

    # Predict the validation set
    y_vali_pred = model.predict(X_val.reshape(X_val.shape[0], -1))

    # Calculate validation accuracy
    val_accuracy = accuracy_score(y_val, y_vali_pred)
    return train_accuracy, val_accuracy

In [ ]:
train_accuracies = []
val_accuracies = []
for i in range(1, 6):  # 5 folds
    train_dir = os.path.join(CROSS_VALI_DIR, f'fold_{i}', 'train')
    val_dir = os.path.join(CROSS_VALI_DIR, f'fold_{i}', 'val')
    train_accuracy, val_accuracy = train_decision_tree(train_dir, val_dir)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    print(f"Training Accuracy for fold {i}: {train_accuracy}")
    print(f"Validation Accuracy for fold {i}: {val_accuracy}")

## 4.3. Semi-Supervised Decision Tree

### 4.3.1. Generating the labeled and unlabeled data for Semi-Supervised Decision Tree

In [ ]:
def load_images_and_split(base_dir, labeled_ratio=0.2):
    """
    Load images from a directory and split them into labeled and unlabeled datasets.

    Args:
    - base_dir (str): Path to the directory containing subdirectories for each class.
    - labeled_ratio (float): The fraction of data to keep as labeled.

    Returns:
    - labeled_data (np.array): Array of labeled data.
    - labeled_labels (list): List of labels for the labeled data.
    - unlabeled_data (np.array): Array of unlabeled data.
    """
    data = []
    labels = []
    classes = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    class_labels = {cls_name: idx for idx, cls_name in enumerate(classes)}

    # Load all data and labels
    for cls_name in classes:
        class_dir = os.path.join(base_dir, cls_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = Image.open(img_path)
            img_data = np.array(img).flatten()
            data.append(img_data)
            labels.append(class_labels[cls_name])

    # Convert to numpy arrays
    data = np.array(data)
    labels = np.array(labels)

    # Split into labeled and unlabeled
    labeled_indices, unlabeled_indices = train_test_split(
        np.arange(len(labels)),
        test_size=1-labeled_ratio,
        random_state= RANDOM_SEED,
        stratify=labels  # Keep the distribution of classes consistent
    )

    labeled_data = data[labeled_indices]
    labeled_labels = labels[labeled_indices]
    unlabeled_data = data[unlabeled_indices]

    return labeled_data, labeled_labels, unlabeled_data

In [ ]:
labeled_data, labeled_labels, unlabeled_data = load_images_and_split(TRAIN_DIR, labeled_ratio=0.2)

print("Labeled data shape:", labeled_data.shape)
print("Unlabeled data shape:", unlabeled_data.shape)

### 4.3.2. Model Training

 *need to add validation*

In [ ]:
# Create the decision tree model
model = DecisionTreeClassifier(random_state = RANDOM_SEED)

# Initial training: Use the labeled data to train the model
model.fit(labeled_data, labeled_labels)

# Threshold list: decrease from 95% to 90% to 85% to 80%
thresholds = [0.95, 0.90, 0.85, 0.80]
test_accuracies = []  # Store the test accuracy for each threshold
train_accuracies = []  # Store the training accuracy for each threshold

# Iterate through each threshold
for confidence_threshold in thresholds:
    pseudo_labels = []  # Reset the pseudo label list for each loop

    # Predict the labels for the unlabeled data
    probabilities = model.predict_proba(unlabeled_data)
    max_probabilities = np.max(probabilities, axis=1)

    # Select high-confidence predictions to use as pseudo labels
    for i, prob in enumerate(max_probabilities):
        if prob > confidence_threshold:
            pseudo_labels.append((unlabeled_data[i], np.argmax(probabilities[i])))

    # If there are enough high-confidence predictions, add them as pseudo labels to the training data
    if pseudo_labels:
        pseudo_data, pseudo_labels = zip(*pseudo_labels)
        new_data = np.vstack([labeled_data] + list(pseudo_data))
        new_labels = np.hstack([labeled_labels] + list(pseudo_labels))

        # Retrain the model with the new training data including pseudo labels
        model.fit(new_data, new_labels)

    # Calculate and store training accuracy
    train_accuracy = model.score(new_data, new_labels)
    train_accuracies.append(train_accuracy)

    # Measure the model's performance on the original labeled test set
    test_accuracy = model.score(test_data, test_labels)
    test_accuracies.append(test_accuracy)
    print(f'Training Accuracy at threshold {confidence_threshold*100}%: {train_accuracy}')
    print(f'Test Accuracy at threshold {confidence_threshold*100}%: {test_accuracy}')


# Plot the test and training accuracies
plt.figure(figsize=(10, 6))
plt.plot([str(th*100) for th in thresholds], test_accuracies, marker='o', linestyle='-', color='b', label='Test Accuracy')
plt.plot([str(th*100) for th in thresholds], train_accuracies, marker='x', linestyle='--', color='r', label='Training Accuracy')
plt.title('Accuracy vs Confidence Threshold')
plt.xlabel('Confidence Threshold (%)')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


## 4.4. Semi-Supervised Decision Tree with 5-fold validation